In [1]:
import geopandas as gpd
import pandas as pd
from scipy.spatial import KDTree
from sklearn.preprocessing import QuantileTransformer
from sklearn.neighbors import KNeighborsClassifier

In [2]:
clusters_dir = '/data/uscuni-ulce/processed_data/clusters/'
chars_dir = '/data/uscuni-ulce/processed_data/chars/'
regions_datadir = "/data/uscuni-ulce/"
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
)

In [3]:
region_id = 69333

In [4]:
def assign_noise(region_id):
    
    # read cluster data and characters
    clusters = pd.read_parquet(f'{clusters_dir}{region_id}_clusters.pq', columns=['final', 'morph'])
    chars = pd.read_parquet(f'{chars_dir}primary_chars_{region_id}.parquet')
    chars = chars[chars.index >= 0]
    assert (clusters.index == chars.index).all()


    is_noise = clusters['final'].values == -1
    # morph -> final
    morph_to_final = pd.Series(clusters.loc[~is_noise, 'final'].values,
                               clusters.loc[~is_noise, 'morph'].values)
    morph_to_final = morph_to_final[~morph_to_final.index.duplicated()]

    ## aggreate morphotope medians and noise in the same dataframe and normalise
    ## cannot use individual tess cells as takes too much time
    morph_medians = chars[~is_noise].groupby(clusters[~is_noise].morph.values).median()
    all_data = pd.concat((morph_medians, chars[is_noise]))
    scalar = QuantileTransformer(subsample=None, output_distribution='uniform')
    vals = scalar.fit_transform(all_data)
    all_data = pd.DataFrame(vals, index=all_data.index, columns=all_data.columns).fillna(0)


    # setup tree data
    query_data = all_data.loc[chars.index[is_noise]]
    tree_data = all_data.loc[morph_medians.index]

    ## query a knn classifier and assign predictions
    tree = KNeighborsClassifier(n_neighbors=9, n_jobs=-1)
    tree = tree.fit(X=tree_data.values, y=morph_to_final.loc[tree_data.index].values)
    predictions = tree.predict(query_data.values)
    clusters.loc[is_noise, 'final'] = predictions
    return clusters.final

In [ ]:
for region_id, _ in region_hulls.iterrows():
    print(region_id)
    final_clusters = assign_noise(region_id)
    clusters = gpd.read_parquet(f'{clusters_dir}{region_id}_clusters.pq')
    clusters['final_without_noise'] = final_clusters
    clusters.to_parquet(f'{clusters_dir}{region_id}_clusters.pq')

4
10
132
134
286
313
400
523
765
801
832
913
960
1124
1154
1387
1478
1515
1605
1718
1736
1782
1970
1981
2096
2322
2350
2478
2514
2625
2728
2975
3039
3109
3150
3221
3250
3526
3610
3612
3701
3705
3752
3759
3981
4070
4214
4215
4235
4284
4356
4382
4723
4805
5096
5191
5246
5310
5408
5427
5662
5671
5766
5883
6254
6529
6560
6576
6741
6749
6811
6873
6996
7068
7094
7280
7485
7528
7534
7681
7688
7712
7727
7805
7914
7937
7963
8046
8216
8238
8256
8265
8345
8374
8396
8592
8707
8731
8757
8759
8813
9016
9064
9074
9150
9169
9194
9284
9824
9924
9954
9972
10019
10086
10095
10124
10179
10222
10263
10277
10455
10510
10511
10563
10579
10602
10666
10794
10847
10908


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (944). n_quantiles is set to n_samples.
  warnings.warn(


10926
10970
11002
11019
11057
11141
11210
11256
11261
11305
11309
11311
11318
11367
11444
11455
11471
11667
11678
11735
11757
11799
11877
11905
12027
12084
12100
12115
12154
12191
12381
12440
12483
12552
12667
12707
12755
12756
12844
12919
12965
13076
13137
13172
13191
13196
13229
13301
13395
13442
13482
13506
13553
13555
13614
13616
13655
13677
13779
13810
13881
13897
13965
14084
14245
14319
14321
14327
14364
14383
14550
14552
14623
14679
14702
14735
14789
14933
15019
15139
15265
15331
15347
15484
15509
15545
15555
15613
15626
15653
15690
15716
15794
15802
15933
15970
16217
16242
16256
16291
16318
16429
16509
16510
16566
16632
16685
16688
16745
16921
17171
17268
17389
17393
17458
17677
17690


In [ ]:
## assign back to file
clusters = gpd.read_parquet(f'{clusters_dir}{region_id}_clusters.pq')
clusters['final'] = final_clusters
# clusters.to_parquet(f'{clusters_dir}{region_id}_clusters.pq')

In [7]:
# %%time
# tree = KDTree(tree_data.values)
# _, idxs = tree.query(query_data.values, k=1, workers=-1)

# clusters.loc[is_noise, 'final'] =  morph_to_final.loc[tree_data.index[idxs]].values

### Plotting

In [8]:
import numpy as np
from core.cluster_validation import get_color


model_params = '_75_0_None_None_False'

def morphotopes_to_etcs(region_id, etcs=True, model_params='_100_0_None_None_False'):


    if etcs:
        etcs = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/tessellations/tessellation_{region_id}.parquet')

    else:
        etcs = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/buildings/buildings_{region_id}.parquet')
        
    etcs['label'] = -1
    
    morphotopes = pd.read_parquet(f'/data/uscuni-ulce/processed_data/morphotopes/tessellation_labels_morphotopes_{region_id}{model_params}.pq')
    morphotopes.loc[:, 'morphotope_label'] =  morphotopes.values[:, 0]

    morph_dict = pd.Series(np.arange(np.unique(morphotopes.values).shape[0]),
                       np.unique(morphotopes.values))
    etcs.loc[morphotopes.index, 'label'] = morphotopes.map(lambda x: morph_dict.loc[x]).values
    etcs['morph'] = str(region_id) + '_' + '-1'
    etcs.loc[morphotopes.index, 'morph'] = str(region_id) + '_' + morphotopes.values
    return etcs

In [9]:
final_colors = pd.DataFrame(get_color(final_clusters.values), final_clusters.values).drop_duplicates()
final_colors.loc[-1] = [255,255,255]

In [10]:
region_id = 69333 

In [11]:
etcs = morphotopes_to_etcs(region_id, etcs=False, model_params=model_params)

In [12]:
etcs['geometry'] = etcs.simplify(1).to_crs(epsg=4326).make_valid()
etcs = etcs[etcs['geometry'].geom_type == 'Polygon']


In [13]:
etcs['final'] = final_clusters

In [14]:
%%time
import lonboard
# plotting = tessellation[tessellation.index.isin(X_train.index)].copy()
layer = lonboard.SolidPolygonLayer.from_geopandas(etcs, opacity=.7)

CPU times: user 1 s, sys: 94.2 ms, total: 1.09 s
Wall time: 1.09 s


In [15]:
from sidecar import Sidecar
sc = Sidecar(title=f'Final Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.DarkMatter)
with sc:
    display(m)

In [16]:
from core.cluster_validation import get_color
layer.get_fill_color = final_colors.loc[etcs.final].values.astype('uint8')